# 1-2

In [12]:
using JuMP, HiGHS
model = Model(HiGHS.Optimizer)
bid_prices = [980, 970, 960, 940]
ask_prices = [990, 985, 972, 954]

payouts = [
    [100, 80, 70, 60],    # Year 1
    [110, 90, 80, 50],    # Year 2
    [1100, 1120, 1090, 1110]  # Year 3
]
income_required = [10000, 11000]
@variable(model, x[1:4] >= 0)
@variable(model, y[1:4] >= 0)
@variable(model, c[0:3] >= 0)
@constraint(model, c[0] == 100000 + sum(bid_prices[i] * y[i] for i in 1:4) - sum(ask_prices[i] * x[i] for i in 1:4))
@constraint(model, c[1] == 1.03 * c[0] + sum(payouts[1][i] * (x[i] - y[i]) for i in 1:4) - income_required[1])
@constraint(model, c[2] == 1.03 * c[1] + sum(payouts[2][i] * (x[i] - y[i]) for i in 1:4) - income_required[2]) 
@constraint(model, c[3] == 1.03 * c[2] + sum(payouts[3][i] * (x[i] - y[i]) for i in 1:4))
@constraint(model, max_sell[i in 1:4], x[i] <= 1000)
@constraint(model, max_buy[i in 1:4], y[i] <= 1000)
@objective(model, Max, c[3])

set_silent(model)
optimize!(model)

println("Max final cash amount: ", objective_value(model))

Max final cash amount: 155789.88297872338


# 2-2

In [30]:
using JuMP
using GLPK

 
demand = [4000, 8000, 20000, 12000, 6000, 2000] 
initial_inventory = 2000  
initial_production = 4000 

model = Model(GLPK.Optimizer)

@variable(model, P[0:6] >= 0)  
@variable(model, V[0:6] >= 0)
@variable(model, D[1:6] >= 0)
@variable(model, I[1:6] >= 0)  
@variable(model, Va[0:6] >= 0)  
@variable(model, Vb[0:6] >= 0)  

@objective(model, Min, 
    sum(0.10 * P[t] + 1.50 * I[t] + 1.00 * D[t] + 0.20 * Va[t] + 0.50 * Vb[t] for t in months)
)


@constraint(model, V[0] == initial_inventory)  
@constraint(model, Va[0] == initial_inventory)
@constraint(model, P[0] == initial_production) 

for t in 1:6
    @constraint(model, V[t-1] + P[t] - demand[t] == V[t])
end

for t in 1:6
    @constraint(model, P[t] ==  P[t-1] + I[t] - D[t])
end

# Inventory tiered storage constraint
for t in 1:6
    @constraint(model, Va[t] + Vb[t] == V[t])
    @constraint(model, Va[t] <= 8000)
end

# Solve the model
optimize!(model)

# Output the results
println("Optimal Production Schedule:")
for t in months
    println("Month $t: Production = ", value(P[t]), ", Inventory = ", value(V[t]))
end
println("Total Cost: ", objective_value(model))

Optimal Production Schedule:
Month 1: Production = 10000.000000000002, Inventory = 8000.0
Month 2: Production = 10666.666666666668, Inventory = 10666.666666666666
Month 3: Production = 10666.666666666668, Inventory = 1333.3333333333333
Month 4: Production = 10666.666666666668, Inventory = 0.0
Month 5: Production = 8000.0, Inventory = 2000.0
Month 6: Production = 8000.0, Inventory = 7999.999999999999
Total Cost: 25266.666666666668


# 2-4

In [35]:
using JuMP
using GLPK

 
demand = [4000, 8000, 20000, 12000, 6000, 2000] 
initial_inventory = 2000  
initial_production = 4000 

model = Model(GLPK.Optimizer)

@variable(model, P[0:6] >= 0)  
@variable(model, V[0:6] >= 0)
@variable(model, D[1:6] >= 0)
@variable(model, I[1:6] >= 0)  
@variable(model, Va[0:6] >= 0)  
@variable(model, Vb[0:6] >= 0)  
@variable(model, b[0:6] >=0) # backtrack from month t + 1
@objective(model, Min, 
    sum(0.10 * P[t] + 1.50 * I[t] + 1.00 * D[t] + 0.20 * Va[t] + 0.50 * Vb[t] + 0.25 * b[t] for t in months)
)


@constraint(model, V[0] == initial_inventory)  
@constraint(model, Va[0] == initial_inventory)
@constraint(model, P[0] == initial_production) 
@constraint(model, b[0] == 0)
for t in 1:6
    @constraint(model, V[t-1] + P[t] + b[t] == V[t] + demand[t] + b[t - 1])
end

for t in 1:6
    @constraint(model, P[t] ==  P[t-1] + I[t] - D[t])
end

# Inventory tiered storage constraint
for t in 1:6
    @constraint(model, Va[t] + Vb[t] == V[t])
    @constraint(model, Va[t] <= 8000)
end

# Solve the model
optimize!(model)

# Output the results
println("Optimal Production Schedule:")
for t in months
    println("Month $t: Production = ", value(P[t]), ", Inventory = ", value(V[t]))
end
println("Total Cost: ", objective_value(model))

Optimal Production Schedule:
Month 1: Production = 8333.333333333334, Inventory = 6333.333333333334
Month 2: Production = 8333.333333333334, Inventory = 6666.666666666667
Month 3: Production = 8333.333333333334, Inventory = 0.0
Month 4: Production = 8333.333333333334, Inventory = 0.0
Month 5: Production = 8333.333333333334, Inventory = 0.0
Month 6: Production = 8333.333333333334, Inventory = 0.0
Total Cost: 19100.000000000004


# 3-2

In [43]:
model = Model(GLPK.Optimizer)
@variable(model, x[1:6])
@variable(model, t[1:6]) # auxiliary variables

@constraint(model, 3x[1] + 3x[2] + 4x[3] + 6x[6] == 15)
@constraint(model, 2x[1] - 6x[2] - x[3] - 5x[4] + 2x[5] - x[6] == 10)
@constraint(model, x[1] - x[2] + x[3] - 3x[5] + x[6] == 2)
@constraint(model, 2x[1] + x[3] - x[4] + 6x[5] == 0)

@objective(model, Min, sum(t[i] for i in 1:6))
# @constraint(model, t[i] >= -x[i] for i in 1:6)
# @constraint(model, t[i] >= x[i] for i in 1:6)
for i in 1:6
    @constraint(model, t[i] >= -x[i])
    @constraint(model, t[i] >= x[i])
end
optimize!(model)
println("Optimal solution:", objective_value(model))

Optimal solution:7.917808219178083


# 3-4

In [46]:
model = Model(GLPK.Optimizer)
@variable(model, x[1:6])
@variable(model, z) # auxiliary variables

@constraint(model, 3x[1] + 3x[2] + 4x[3] + 6x[6] == 15)
@constraint(model, 2x[1] - 6x[2] - x[3] - 5x[4] + 2x[5] - x[6] == 10)
@constraint(model, x[1] - x[2] + x[3] - 3x[5] + x[6] == 2)
@constraint(model, 2x[1] + x[3] - x[4] + 6x[5] == 0)

@objective(model, Min, z)
for i in 1:6
    @constraint(model, z >= -x[i])
    @constraint(model, z >= x[i])
end
optimize!(model)
println("Optimal solution:", objective_value(model))

Optimal solution:3.105527638190955


# 4-2

In [48]:
using JuMP
using GLPK

T = 1:18  
d = [2, 16, 9, 8, 10, 6, 2, 2, 9, 5, 3, 2, 1, 7, 4, 3, 9, 1]  

P = [
    (1, 2), (2, 3), (2, 4), (3, 5), (4, 6), (4, 7), (5, 6), (6, 8), (6, 9), (6, 10), 
    (6, 11), (9, 12), (7, 13), (2, 14), (4, 15), (14, 15), (8, 16), (11, 16), (14, 16), 
    (12, 17), (17, 18)
]

model = Model(GLPK.Optimizer)
@variable(model, s[T] >= 0) 
@variable(model, z >= 0)
@objective(model, Min, z)

# Precedence constraints
for (i, j) in P
    @constraint(model, s[j] >= s[i] + d[i])
end

# Project completion time constraints
for t in T
    @constraint(model, z >= s[t] + d[t])
end

optimize!(model)
println("Optimal Start Times:")
for t in T
    println("Task $t: Start Time = ", value(s[t]))
end
println("Minimum Project Completion Time: ", value(z))

Optimal Start Times:
Task 1: Start Time = 0.0
Task 2: Start Time = 2.0
Task 3: Start Time = 18.0
Task 4: Start Time = 18.0
Task 5: Start Time = 27.0
Task 6: Start Time = 37.0
Task 7: Start Time = 26.0
Task 8: Start Time = 43.0
Task 9: Start Time = 43.0
Task 10: Start Time = 43.0
Task 11: Start Time = 43.0
Task 12: Start Time = 52.0
Task 13: Start Time = 28.0
Task 14: Start Time = 53.0
Task 15: Start Time = 60.0
Task 16: Start Time = 60.0
Task 17: Start Time = 54.0
Task 18: Start Time = 63.0
Minimum Project Completion Time: 64.0


# 4-4

In [50]:
using JuMP
using GLPK

# Define the data
T = 1:18  # Set of tasks
d = [2, 16, 9, 8, 10, 6, 2, 2, 9, 5, 3, 2, 1, 7, 4, 3, 9, 1]  # Durations of tasks
m = [0, 3, 1, 2, 2, 1, 1, 0, 2, 1, 1, 0, 0, 2, 2, 1, 3, 0]  # Maximum reduction per task
c = [0, 5, 3, 4, 6, 2, 1, 0, 7, 3, 2, 0, 0, 4, 5, 2, 6, 0]  # Cost of reducing each task by one week
R = 30  # Weekly bonus (in thousands of dollars)
χ_star = 64  # Minimum completion time without reductions (from Problem 4-2)

# Precedence constraints (i, j) where task i must finish before task j starts
P = [
    (1, 2), (2, 3), (2, 4), (3, 5), (4, 6), (4, 7), (5, 6), (6, 8), (6, 9), (6, 10), 
    (6, 11), (9, 12), (7, 13), (2, 14), (4, 15), (14, 15), (8, 16), (11, 16), (14, 16), 
    (12, 17), (17, 18)
]

model = Model(GLPK.Optimizer)
@variable(model, s[T] >= 0)  # Start time of each task
@variable(model, n[T] >= 0)  # Number of weeks each task is reduced
@variable(model, z >= 0)  # Number of weeks the project is completed early

@objective(model, Max, R * z - sum(c[t] * n[t] for t in T))

# Precedence constraints
for (i, j) in P
    @constraint(model, s[j] >= s[i] + (d[i] - n[i]))
end
# Project completion time constraints
for t in T
    @constraint(model, χ_star - z >= s[t] + (d[t] - n[t]))
end


# Task reduction limits
for t in T
    @constraint(model, n[t] <= m[t])
end

# Solve the model
optimize!(model)

# # Results
# println("Optimal Start Times:")
# for t in T
#     println("Task $t: Start Time = ", value(s[t]), ", Reduction = ", value(n[t]))
# end
println("Project Completion Time: ", value(χ_star - z))
println("Weeks Completed Early: ", value(z))
println("Maximum Profit: \$", objective_value(model))

Project Completion Time: 52.0
Weeks Completed Early: 12.0
Maximum Profit: $296.0


# 5-2

In [53]:
using JuMP
using GLPK

restaurants = ["Short Stack", "Great Dane", "Old Fashioned", "Red Sushi", "Ian’s at Garver", "Weary Traveler", "Alchemy"]
n = length(restaurants)  # Number of restaurants

E = [
    ("Short Stack", "Great Dane", 4),
    ("Short Stack", "Old Fashioned", 6),
    ("Short Stack", "Red Sushi", 5),
    ("Great Dane", "Old Fashioned", 7),
    ("Great Dane", "Weary Traveler", 2),
    ("Old Fashioned", "Red Sushi", 5.5),
    ("Old Fashioned", "Ian’s at Garver", 4),
    ("Red Sushi", "Ian’s at Garver", 5),
    ("Ian’s at Garver", "Alchemy", 1),
    ("Weary Traveler", "Alchemy", 6),
    ("Weary Traveler", "Ian’s at Garver", 8)
]

A = zeros(Int, n, length(E))  # Initialize incidence matrix
for (idx, (i, j, c)) in enumerate(E)
    i_idx = findfirst(isequal(i), restaurants)
    j_idx = findfirst(isequal(j), restaurants)
    A[i_idx, idx] = 1  # Edge leaves node i
    A[j_idx, idx] = -1  # Edge enters node j
end

b = zeros(n)
start_idx = findfirst(isequal("Short Stack"), restaurants)
end_idx = findfirst(isequal("Alchemy"), restaurants)
b[start_idx] = 1  # Net flow at start node = +1
b[end_idx] = -1   # Net flow at end node = -1

# Create the model
model = Model(GLPK.Optimizer)

# Decision variables: x[e] = 1 if the path includes edge e
@variable(model, x[1:length(E)], Bin)

c = [c for (i, j, c) in E]  # Vector of driving times
@objective(model, Min, c' * x)

# Flow conservation constraints: A x = b
@constraint(model, A * x .== b)

optimize!(model)

println("Shortest Path Trips:")
for (idx, (i, j, c)) in enumerate(E)
    if value(x[idx]) == 1
        println("From $i to $j: $c minutes")
    end
end
println("Total Driving Time: ", objective_value(model), " minutes")

Shortest Path Trips:
From Short Stack to Old Fashioned: 6 minutes
From Old Fashioned to Ian’s at Garver: 4 minutes
From Ian’s at Garver to Alchemy: 1 minutes
Total Driving Time: 11.0 minutes
